# JEM092 Asset Pricing
# Seminar 9
## Lukáš Petrásek
### Charles University
### lukas.petrasek@fsv.cuni.cz
## 19.4.2022

In [ ]:
# Import third-party packages.
library(PerformanceAnalytics)
library(portsort)
library(quantmod)
library(readr)
library(sandwich)
library(xts)

# Data preparation

In [ ]:
# Load tickers.
sp100_tickers <- read_csv(
    '/home/lukas/projects/asset-pricing/summer-semester-2022/seminar_9/sp100_tickers.csv',
    col_names = FALSE,
)

In [ ]:
START_DATE <- '2011-01-01'
END_DATE <- '2021-03-31'

# Load stock prices and nominals.
prices <- c()
nominals <- c()
for (i in 1:nrow(sp100_tickers)) {
    ticker <- as.character(sp100_tickers[i, 1])
    stock_data <- getSymbols(
        ticker,
        from = START_DATE,
        to = END_DATE,
        src = 'yahoo',
        warnings = FALSE,
        auto.assign = FALSE
    )

    stock_prices = stock_data[, 6]
    stock_nominals = stock_data[, 5] * stock_prices
    colnames(stock_prices) <- ticker
    colnames(stock_nominals) <- ticker
    prices <- cbind(prices, stock_prices)
    nominals <- cbind(nominals, stock_nominals)
}

In [ ]:
prices[1:5, 1:3]
nominals[1:5, 1:3]

In [ ]:
# Compute daily and quarterly returns.
daily_returns <- lapply(prices, dailyReturn, USE.NAMES = TRUE)
quarterly_returns <- lapply(prices, quarterlyReturn, USE.NAMES = TRUE)
daily_returns <- do.call('cbind', daily_returns)
quarterly_returns <- do.call('cbind', quarterly_returns)
colnames(daily_returns) <- paste('dr', colnames(prices), sep = '_')
colnames(quarterly_returns) <- paste('qr', colnames(prices), sep = '_')
index(quarterly_returns) <- as.Date(as.yearqtr(index(quarterly_returns), format = "Q%q/%y"), frac = 1)

# Compute quarterly volatilities.
quarterly_volatilities <- as.xts(aggregate(daily_returns, as.yearqtr(as.yearmon(time(daily_returns))), sd))
index(quarterly_volatilities) <- as.Date(as.yearqtr(index(quarterly_volatilities), format = "Q%q/%y"), frac = 1)
colnames(quarterly_volatilities) <- paste('qv', colnames(prices), sep = '_')

# Comupte quarterly nominals.
quarterly_nominals <- as.xts(aggregate(nominals, as.yearqtr(as.yearmon(time(nominals))), sum))
index(quarterly_nominals) <- as.Date(as.yearqtr(index(quarterly_nominals), format = "Q%q/%y"), frac = 1)
colnames(quarterly_nominals) <- paste('qn', colnames(prices), sep = '_')

In [ ]:
# Combine the quarterly data.
head(cbind(quarterly_returns, quarterly_volatilities, quarterly_nominals)[, 96:100])
head(cbind(quarterly_returns, quarterly_volatilities, quarterly_nominals)[, 193:197])

# Fama-MacBeth regressions

Procedure:

1) Run periodic cross-sectional regressions of the dependent variable ($Y$) on one or more independent variables ($X_1$, $X_2$, ...) using data from each time period $t$.

→ We obtain intercept and slope coefficients on each independent variable for each period.

2) Analyze the time series of each of the regression coefficients to determine whether the average coefficient differs from zero.

## Fama-MacBeth regressions of returns on nominals

In [ ]:
# Initialize an xts object for storing regression results.
fm_cs_results_nominals <- as.xts(
    matrix(
        nrow = nrow(quarterly_nominals) - 1,
        ncol = 5
    ),
    order.by = index(quarterly_nominals[2:nrow(quarterly_nominals)])
)
names(fm_cs_results_nominals) <- c('Intercept', 'Nominal', 'R^2', 'Adjusted R^2', 'N')

# Iterate over rows and perform cross-sectional regressions.
for (i in 1:nrow(fm_cs_results_nominals)) {
    # Avoid look-ahead bias by sorting the returns after the quarter used to compute breakpoints ends.
    quarter_nominals <- quarterly_nominals[i]
    quarter_returns <- quarterly_returns[i + 1]

    # Save regression coefficients and other statistics.
    model <- lm(t(quarter_returns) ~ 1 + t(quarter_nominals))
    fm_cs_results_nominals[i, 1] <- model$coefficients[[1]]
    fm_cs_results_nominals[i, 2] <- model$coefficients[[2]]
    fm_cs_results_nominals[i, 3] <- summary(model)$r.squared
    fm_cs_results_nominals[i, 4] <- summary(model)$adj.r.squared
    fm_cs_results_nominals[i, 5] <- nobs(model)
}

fm_cs_results_nominals

In [ ]:
# Compute time series means, standard errors.
fm_results_nominals <- as.data.frame(matrix(nrow = 2, ncol = 2))
names(fm_results_nominals) <- c('Intercept', 'Nominal')
for (i in 1:ncol(fm_results_nominals)) {
    model <- lm(fm_cs_results_nominals[, i] ~ 1)
    fm_results_nominals[1, i] <- model$coefficients[[1]]
    fm_results_nominals[2, i] <- model$coefficients[[1]] / sqrt(NeweyWest(model, lag = 4))[[1]]
}

fm_results_nominals

## Fama-MacBeth regressions of returns on volatilities

In [ ]:
# Initialize an xts object for storing regression results.
fm_cs_results_volatilities <- as.xts(
    matrix(
        nrow = nrow(quarterly_volatilities) - 1,
        ncol = 5
    ),
    order.by = index(quarterly_volatilities[2:nrow(quarterly_volatilities)])
)
names(fm_cs_results_volatilities) <- c('Intercept', 'Volatility', 'R^2', 'Adjusted R^2', 'N')

# Iterate over rows and perform cross-sectional regressions.
for (i in 1:nrow(fm_cs_results_volatilities)) {
    # Avoid look-ahead bias by sorting the returns after the quarter used to compute breakpoints ends.
    quarter_volatilities <- quarterly_volatilities[i]
    quarter_returns <- quarterly_returns[i + 1]

    # Save regression coefficients and other statistics.
    model <- lm(t(quarter_returns) ~ 1 + t(quarter_volatilities))
    fm_cs_results_volatilities[i, 1] <- model$coefficients[[1]]
    fm_cs_results_volatilities[i, 2] <- model$coefficients[[2]]
    fm_cs_results_volatilities[i, 3] <- summary(model)$r.squared
    fm_cs_results_volatilities[i, 4] <- summary(model)$adj.r.squared
    fm_cs_results_volatilities[i, 5] <- nobs(model)
}

# Compute time series means, standard errors.
fm_results_volatilities <- as.data.frame(matrix(nrow = 2, ncol = 2))
names(fm_results_volatilities) <- c('Intercept', 'Volatility')
for (i in 1:ncol(fm_results_volatilities)) {
    model <- lm(fm_cs_results_volatilities[, i] ~ 1)
    fm_results_volatilities[1, i] <- model$coefficients[[1]]
    fm_results_volatilities[2, i] <- model$coefficients[[1]] / sqrt(NeweyWest(model, lag = 4))[[1]]
}

fm_results_volatilities

## Fama-MacBeth regressions of returns on nominals and volatilities

In [ ]:
# Initialize an xts object for storing regression results.
fm_cs_results_nominals_volatilities <- as.xts(
    matrix(
        nrow = nrow(quarterly_volatilities) - 1,
        ncol = 6
    ),
    order.by = index(quarterly_volatilities[2:nrow(quarterly_volatilities)])
)
names(fm_cs_results_nominals_volatilities) <- c('Intercept', 'Nominal', 'Volatility', 'R^2', 'Adjusted R^2', 'N')

# Iterate over rows and perform cross-sectional regressions.
for (i in 1:nrow(fm_cs_results_nominals_volatilities)) {
    # Avoid look-ahead bias by sorting the returns after the quarter used to compute breakpoints ends.
    quarter_nominals <- quarterly_nominals[i]
    quarter_volatilities <- quarterly_volatilities[i]
    quarter_returns <- quarterly_returns[i + 1]

    # Save regression coefficients and other statistics.
    model <- lm(t(quarter_returns) ~ 1 + t(quarter_nominals) + t(quarter_volatilities))
    fm_cs_results_nominals_volatilities[i, 1] <- model$coefficients[[1]]
    fm_cs_results_nominals_volatilities[i, 2] <- model$coefficients[[2]]
    fm_cs_results_nominals_volatilities[i, 3] <- model$coefficients[[3]]
    fm_cs_results_nominals_volatilities[i, 4] <- summary(model)$r.squared
    fm_cs_results_nominals_volatilities[i, 5] <- summary(model)$adj.r.squared
    fm_cs_results_nominals_volatilities[i, 6] <- nobs(model)
}

# Compute time series means, standard errors.
fm_results_nominals_volatilities <- as.data.frame(matrix(nrow = 2, ncol = 3))
names(fm_results_nominals_volatilities) <- c('Intercept', 'Nominal', 'Volatility')
for (i in 1:ncol(fm_results_nominals_volatilities)) {
    model <- lm(fm_cs_results_nominals_volatilities[, i] ~ 1)
    fm_results_nominals_volatilities[1, i] <- model$coefficients[[1]]
    fm_results_nominals_volatilities[2, i] <- model$coefficients[[1]] / sqrt(NeweyWest(model, lag = 4))[[1]]
}

fm_results_nominals_volatilities